---
## create TFrecord file:

In [1]:
import os
import numpy as np
import tensorflow as tf

/home/mtx/local/anaconda2/envs/tf1.5/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
dirbase='/nfs/P100/SDSSV_Classifiers/processed_dataset/TF_dataset'

Define corresponding data type.

In [3]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

Rewrite the following functions for different dataset.

In [4]:
def convert_to(data_set, Dir, name):
    """Converts a dataset to tfrecords."""
    features = data_set.features
    labels = data_set.labels
    num_examples = data_set.num_examples

    if features.shape[0] != num_examples:
        raise ValueError('Features size %d does not match label size %d.' %
                         (features.shape[0], num_examples))

    filename = os.path.join(Dir, name + '.tfrecords')
    print('Writing', filename)
    writer = tf.python_io.TFRecordWriter(filename)
    for index in xrange(num_examples):
        data_set.index = index
        feature_raw = features[index].reshape(-1).tolist()
        label_raw = labels[index].reshape(-1).tolist()
        example = tf.train.Example(features=tf.train.Features(feature={
            'index': _int64_feature(data_set.index),
            'label_raw': _float_feature(label_raw),
            'feature_raw': _float_feature(feature_raw)}))
        writer.write(example.SerializeToString())
    writer.close()

In [5]:
def main(data, filename):
    # Get the data.
    class data_set():
        pass
    Len = data.shape[0]
    features = data['flux_norm']
    labels = data['label'].reshape(-1,1)
  
    data_set.features = features
    data_set.labels = labels
    data_set.num_examples = Len

    convert_to(data_set, Dir=dirbase, name=filename)

Run the convert code.

In [6]:
star_type = ['wdsb2', 'wd', 'fgkm', 'hotstars', 'yso', 'cv']
Dir = '/nfs/P100/SDSSV_Classifiers/processed_dataset/dataset'
mean = np.load('/nfs/P100/SDSSV_Classifiers/processed_dataset/Norm_mu.npy')
std = np.load('/nfs/P100/SDSSV_Classifiers/processed_dataset/Norm_std.npy')
def loaddata(mode='train'):
    DATA = []
    for i in star_type:
        filename=os.path.join(Dir, mode+'_'+i+'.npy')
        data = np.load(filename)
        data['flux_norm'] = (data['flux_norm']-mean)/std #!!!
        DATA.append(data)
    DATA = np.hstack(DATA)
    random_ind = np.random.permutation(np.arange(DATA.shape[0]))
    DATA = DATA[random_ind]
    print mode, DATA['flux_norm'].mean(), DATA['flux_norm'].std()
    return DATA

In [7]:
train_dataset = loaddata('train')
valid_dataset = loaddata('valid')
test_dataset = loaddata('test')
Dir2 = '/nfs/P100/SDSSV_Classifiers/processed_dataset/TF_dataset'
np.save(os.path.join(Dir2,'train.npy'),train_dataset)
np.save(os.path.join(Dir2,'valid.npy'),valid_dataset)
np.save(os.path.join(Dir2,'test.npy'),test_dataset)

train -0.03525323 2.3771825
valid -0.0004990322 1.1612086
test 0.002460175 3.8226826


In [8]:
main(train_dataset,'training')
main(valid_dataset,'valid')
main(test_dataset,'test')

('Writing', '/nfs/P100/SDSSV_Classifiers/processed_dataset/TF_dataset/training.tfrecords')
('Writing', '/nfs/P100/SDSSV_Classifiers/processed_dataset/TF_dataset/valid.tfrecords')
('Writing', '/nfs/P100/SDSSV_Classifiers/processed_dataset/TF_dataset/test.tfrecords')


**check repeat**

In [23]:
print test_dataset.shape
print valid_dataset.shape
print train_dataset.shape
a = valid_dataset
b = train_dataset

(1778,)
(1779,)
(40506,)


In [27]:
s = np.unique(b)

In [29]:
print s.dtype
print s.shape

[('index', '<i4'), ('label', '<i4'), ('flux_norm', '<f4', (3000,))]
(14250,)


In [47]:
for i in xrange(len(a)):
    for j in xrange(len(s)):
        if a[i]['index'] != s[j]['index']:
            continue
        if a[i]['label'] != s[j]['label']:
            continue
        if (a[i]['flux_norm'][:20] == s[j]['flux_norm'][:20]).sum() == 0:
            continue
        print i,j
print 'finished'

finished
